In [217]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import pairwise_distances
import plotly.express as px
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score as ss


import joblib
import warnings
warnings.filterwarnings(action='ignore') 

# Train
## Data Load -> Scailing -> NMF -> DBSCAN -> get Best EPS, Min_samples

In [218]:
train_df = pd.read_csv("./dataset/train_data.csv")


In [219]:
features = train_df.columns[:-1]
features

Index(['air_inflow', 'air_end_temp', 'out_pressure', 'motor_current',
       'motor_rpm', 'motor_temp', 'motor_vibe'],
      dtype='object')

In [220]:
X_train = train_df[features]
y_train = train_df[['type']]


In [221]:
sc = MinMaxScaler()
X_train[features]= sc.fit_transform(X_train)
X_train.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe
0,0.213922,0.272396,0.0,0.227873,0.202708,0.275531,0.048241
1,0.448217,0.827513,0.0,0.495950,0.821413,0.813992,0.089447


In [222]:
train_df = X_train
train_df['type'] = y_train
train_df.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,0.213922,0.272396,0.0,0.227873,0.202708,0.275531,0.048241,0
1,0.448217,0.827513,0.0,0.495950,0.821413,0.813992,0.089447,0


In [223]:
g1 = ['air_inflow','motor_current']
g2 = ['air_end_temp', 'motor_rpm', 'motor_temp']
g3 = ['motor_vibe', 'out_pressure']

In [224]:
g1_train = train_df[g1]
g2_train = train_df[g2]
g3_train = train_df[g3]
y_train = train_df[['type']]
combined_train = [g1_train, g2_train, g3_train]

In [225]:
svd= NMF(n_components=1, init='random', random_state=0)
svd_list = []
for i in combined_train:
    result = svd.fit_transform(i)
    svd_list.append(result)
#     total_var = svd.explained_variance_ratio_.sum() * 100
#     print(f"{i}total_var : {total_var}")
    
flat_svd_list = []
for i in svd_list:
    result = [j for sub in i for j in sub]
    flat_svd_list.append(result)
svd_train_df = pd.DataFrame({"PC1": flat_svd_list[0], "PC2": flat_svd_list[1], "PC3": flat_svd_list[2]})
svd_train_df = pd.concat([svd_train_df, y_train], axis=1)
svd_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463 entries, 0 to 2462
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     2463 non-null   float64
 1   PC2     2463 non-null   float64
 2   PC3     2463 non-null   float64
 3   type    2463 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 77.1 KB


In [226]:
tdf0_train = svd_train_df[svd_train_df.type == 0]
tdf1_train = svd_train_df[svd_train_df.type == 1]
tdf2_train = svd_train_df[svd_train_df.type == 2]
tdf3_train = svd_train_df[svd_train_df.type == 3]
tdf4_train = svd_train_df[svd_train_df.type == 4]
tdf5_train = svd_train_df[svd_train_df.type == 5]
tdf6_train = svd_train_df[svd_train_df.type == 6]
tdf7_train = svd_train_df[svd_train_df.type == 7]
tdf_train = [tdf0_train, tdf1_train, tdf2_train, tdf3_train, tdf4_train, tdf5_train, tdf6_train, tdf7_train]

In [227]:
def draw_chart(df):
    fig = px.scatter_3d(
        df, x='PC1', y='PC2', z='PC3', color='type',symbol='type', opacity=0.5, size_max=10,height=600
    )
    fig.show()

In [228]:
# for tdf in tdf_train:
#     print(tdf.type.unique())
#     draw_chart(tdf)

In [229]:
epsilons = np.linspace(0.01, 2, num=50)
epsilons

array([0.01      , 0.05061224, 0.09122449, 0.13183673, 0.17244898,
       0.21306122, 0.25367347, 0.29428571, 0.33489796, 0.3755102 ,
       0.41612245, 0.45673469, 0.49734694, 0.53795918, 0.57857143,
       0.61918367, 0.65979592, 0.70040816, 0.74102041, 0.78163265,
       0.8222449 , 0.86285714, 0.90346939, 0.94408163, 0.98469388,
       1.02530612, 1.06591837, 1.10653061, 1.14714286, 1.1877551 ,
       1.22836735, 1.26897959, 1.30959184, 1.35020408, 1.39081633,
       1.43142857, 1.47204082, 1.51265306, 1.55326531, 1.59387755,
       1.6344898 , 1.67510204, 1.71571429, 1.75632653, 1.79693878,
       1.83755102, 1.87816327, 1.91877551, 1.95938776, 2.        ])

In [230]:
min_samples = np.arange(2, 20, step=1)
min_samples

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])

In [231]:
epsilons = epsilons
min_samples = min_samples
import itertools
combinations = list(itertools.product(epsilons, min_samples))
N = len(combinations)

In [232]:
def get_scores_and_labels(combinations, X):
  scores = []
  all_labels_list = []

  for i, (eps, num_samples) in enumerate(combinations):
    dbscan_cluster_model = DBSCAN(eps=eps, min_samples=num_samples).fit(X)
    labels = dbscan_cluster_model.labels_
    labels_set = set(labels)
    num_clusters = len(labels_set)
    if -1 in labels_set:
      num_clusters -= 1
    
    if (num_clusters < 2) or (num_clusters > 50):
      scores.append(-10)
      all_labels_list.append('bad')
      c = (eps, num_samples)
      print(f"Combination {c} on iteration {i+1} of {N} has {num_clusters} clusters. Moving on")
      continue
    try:
        scores.append(ss(X, labels))
        all_labels_list.append(labels)
        print(f"Index: {i}, Score: {scores[-1]}, Labels: {all_labels_list[-1]}, NumClusters: {num_clusters}")
    except:
        scores.append('ss Failure')

  best_index = np.argmax(scores)
  best_parameters = combinations[best_index]
  best_labels = all_labels_list[best_index]
  best_score = scores[best_index]

  return {'best_epsilon': best_parameters[0],
          'best_min_samples': best_parameters[1], 
          'best_labels': best_labels,
          'best_score': best_score}

In [233]:
results = []

for tdf in tdf_train:
    X_train = tdf.iloc[:,:-1]
    y_train = tdf[['type']]
    
    result = get_scores_and_labels(combinations, tdf)
    results.append(result)
    
    tdf['cluster'] = result['best_labels']
    
    print(tdf.type.unique())
#     fig = px.scatter_3d(
#     tdf, x='PC1', y='PC2', z='PC3', color='cluster',symbol='cluster', opacity=0.5, size_max=10,height=600,
#     )
#     fig.show()
    print("="*70)
    

Index: 0, Score: 0.32534777510610696, Labels: [0 1 0 2 0 0 1 3 0 1 4 0 2 1 3 1 0 0 1 0 0 0 1 1 0 3 0 1 0 0 0 3 1 0 0 3 0
 4 0 2 3 0 0 0 0 3 0 3 0 0 0 3 0 3 0 1 1 0 1 0 0 3 0 0 1 0 0 0 0 0 0 0 1 0
 3 0 3 0 1 0 1 0 0 3 0 1 3 0 1 0 0 1 0 3 0 0 0 3 1 3 0 1 1 0 1 0 0 1 1 2 4
 3 1 1 3 0 0 1 1 0 0 0 0 3 0 3 0 0 4 0 1 3 0 1 0 2 1 0 1 0 0 3 0 1 0 4 0 0
 0 3 0 3 3 0 0 0 3 0 3 0 1 3 1 3 0 0 0 3 0 0 1 0 0 0 0 0 0 3 1 3 0 3 0 0 0
 1 3 0 2 0 0 1 0 1 0 3 0 2 3 1 0 1 0 0 1 0 1 0 3 1 0 0 0 0 3 1 0 0 2 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 3 3 1 0 3 2 1 1 0 0 2 2 3 3 1 0 0 0 1 1 0 3 0 3 0
 1 4 0 0 0 2 3 0 1 0 1 0 1 0 1 1 0 1 3 0 0 0 1 1 1 0 1 2 1 1 2 1 0 1 0 1 1
 3 3 1 3 0 1 0 1 3 0 0 1 0 0 1 0 0 0 0 3 1 3 1 3 0 0 0 0 0 0 0 3 3 0 0 2 3
 0 0 3 2 1 0 3 0 0 0 1 1 3 0 3 0 3 0 3 0 1 3 2 0 0 0 0 2 3 0 3 3 1 0 0 1 0
 1 3 0 0 0 1 3 0 0 3 0 0 0 0 0 0 3 2 0 0 3 0 3 0 1 0 2 3 0 3 3 0 1 2 0 1 1
 1 0 0 1 0 0 1 1 3 1 3 2 0 3 0 0 1 0 1 0 0 0 1 0 0], NumClusters: 5
Index: 1, Score: 0.32534777510610696, Labels: [0 1 0 2 0 0 1 

Index: 13, Score: -0.316827218551541, Labels: [-1 -1 -1 -1 -1 -1 -1  2  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1  0 -1 -1 -1 -1 -1 -1  1  1  2 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1  1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  0 -1  1 -1 -1 -1 -1 -1 -1 -1  2 -1  1 -1 -1 -1  0 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  2 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1
 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -

Combination (0.17244897959183675, 10) on iteration 81 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 11) on iteration 82 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 12) on iteration 83 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 13) on iteration 84 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 14) on iteration 85 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 15) on iteration 86 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 16) on iteration 87 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 17) on iteration 88 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 18) on iteration 89 of 900 has 1 clusters. Moving on
Combination (0.17244897959183675, 19) on iteration 90 of 900 has 1 clusters. Moving on
Combination (0.21306122448979592, 2) on iteration 91 of 900 has 1 clusters. Moving on
Combination (0.21306122448979592, 3) on iter

Combination (0.41612244897959183, 11) on iteration 190 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 12) on iteration 191 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 13) on iteration 192 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 14) on iteration 193 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 15) on iteration 194 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 16) on iteration 195 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 17) on iteration 196 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 18) on iteration 197 of 900 has 1 clusters. Moving on
Combination (0.41612244897959183, 19) on iteration 198 of 900 has 1 clusters. Moving on
Combination (0.45673469387755106, 2) on iteration 199 of 900 has 1 clusters. Moving on
Combination (0.45673469387755106, 3) on iteration 200 of 900 has 1 clusters. Moving on
Combination (0.45673469387755106, 

Combination (0.6191836734693877, 16) on iteration 285 of 900 has 1 clusters. Moving on
Combination (0.6191836734693877, 17) on iteration 286 of 900 has 1 clusters. Moving on
Combination (0.6191836734693877, 18) on iteration 287 of 900 has 1 clusters. Moving on
Combination (0.6191836734693877, 19) on iteration 288 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 2) on iteration 289 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 3) on iteration 290 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 4) on iteration 291 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 5) on iteration 292 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 6) on iteration 293 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 7) on iteration 294 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 8) on iteration 295 of 900 has 1 clusters. Moving on
Combination (0.659795918367347, 9) on iteration 296 of 90

...

Combination (0.8628571428571429, 11) on iteration 388 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 12) on iteration 389 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 13) on iteration 390 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 14) on iteration 391 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 15) on iteration 392 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 16) on iteration 393 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 17) on iteration 394 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 18) on iteration 395 of 900 has 1 clusters. Moving on
Combination (0.8628571428571429, 19) on iteration 396 of 900 has 1 clusters. Moving on
Combination (0.9034693877551021, 2) on iteration 397 of 900 has 1 clusters. Moving on
Combination (0.9034693877551021, 3) on iteration 398 of 900 has 1 clusters. Moving on
Combination (0.9034693877551021, 4) on iterat

Combination (1.1471428571428572, 11) on iteration 514 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 12) on iteration 515 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 13) on iteration 516 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 14) on iteration 517 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 15) on iteration 518 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 16) on iteration 519 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 17) on iteration 520 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 18) on iteration 521 of 900 has 1 clusters. Moving on
Combination (1.1471428571428572, 19) on iteration 522 of 900 has 1 clusters. Moving on
Combination (1.1877551020408164, 2) on iteration 523 of 900 has 1 clusters. Moving on
Combination (1.1877551020408164, 3) on iteration 524 of 900 has 1 clusters. Moving on
Combination (1.1877551020408164, 4) on iterat

Combination (1.3908163265306124, 16) on iteration 627 of 900 has 1 clusters. Moving on
Combination (1.3908163265306124, 17) on iteration 628 of 900 has 1 clusters. Moving on
Combination (1.3908163265306124, 18) on iteration 629 of 900 has 1 clusters. Moving on
Combination (1.3908163265306124, 19) on iteration 630 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 2) on iteration 631 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 3) on iteration 632 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 4) on iteration 633 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 5) on iteration 634 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 6) on iteration 635 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 7) on iteration 636 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 8) on iteration 637 of 900 has 1 clusters. Moving on
Combination (1.4314285714285715, 9) on iteration 6

Combination (1.6751020408163266, 2) on iteration 739 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 3) on iteration 740 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 4) on iteration 741 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 5) on iteration 742 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 6) on iteration 743 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 7) on iteration 744 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 8) on iteration 745 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 9) on iteration 746 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 10) on iteration 747 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 11) on iteration 748 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 12) on iteration 749 of 900 has 1 clusters. Moving on
Combination (1.6751020408163266, 13) on iteration 7

Combination (1.9187755102040818, 12) on iteration 857 of 900 has 1 clusters. Moving on
Combination (1.9187755102040818, 13) on iteration 858 of 900 has 1 clusters. Moving on
Combination (1.9187755102040818, 14) on iteration 859 of 900 has 1 clusters. Moving on
Combination (1.9187755102040818, 15) on iteration 860 of 900 has 1 clusters. Moving on
Combination (1.9187755102040818, 16) on iteration 861 of 900 has 1 clusters. Moving on
Combination (1.9187755102040818, 17) on iteration 862 of 900 has 1 clusters. Moving on
Combination (1.9187755102040818, 18) on iteration 863 of 900 has 1 clusters. Moving on
Combination (1.9187755102040818, 19) on iteration 864 of 900 has 1 clusters. Moving on
Combination (1.9593877551020409, 2) on iteration 865 of 900 has 1 clusters. Moving on
Combination (1.9593877551020409, 3) on iteration 866 of 900 has 1 clusters. Moving on
Combination (1.9593877551020409, 4) on iteration 867 of 900 has 1 clusters. Moving on
Combination (1.9593877551020409, 5) on iterati

In [234]:
results[0]

{'best_epsilon': 0.01,
 'best_min_samples': 4,
 'best_labels': array([0, 1, 2, 3, 2, 2, 1, 4, 2, 1, 5, 0, 3, 6, 7, 1, 2, 0, 6, 2, 2, 0,
        1, 6, 0, 7, 0, 6, 0, 2, 0, 7, 6, 0, 0, 4, 0, 5, 2, 3, 7, 0, 2, 0,
        0, 4, 0, 7, 2, 2, 2, 7, 0, 4, 2, 1, 1, 0, 1, 2, 2, 7, 0, 0, 6, 0,
        2, 0, 0, 0, 0, 2, 6, 2, 4, 2, 4, 2, 1, 0, 6, 2, 2, 4, 0, 1, 4, 0,
        1, 0, 0, 6, 2, 7, 0, 2, 2, 7, 1, 4, 0, 6, 6, 2, 1, 0, 0, 1, 6, 3,
        5, 4, 6, 6, 4, 2, 0, 1, 1, 2, 0, 0, 0, 4, 0, 4, 2, 2, 5, 0, 6, 4,
        2, 6, 0, 3, 6, 0, 1, 0, 2, 7, 2, 1, 0, 5, 0, 0, 0, 7, 2, 4, 7, 0,
        0, 0, 7, 2, 7, 2, 1, 7, 6, 4, 0, 2, 2, 7, 2, 2, 6, 2, 0, 0, 0, 0,
        2, 4, 1, 7, 2, 7, 2, 0, 0, 6, 4, 2, 3, 0, 0, 6, 2, 1, 2, 7, 2, 3,
        4, 6, 2, 6, 0, 0, 1, 0, 1, 2, 4, 6, 2, 0, 2, 0, 4, 6, 0, 2, 3, 2,
        0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 6, 2, 0, 2, 7, 4, 6, 2, 7, 3, 1, 6,
        0, 0, 3, 3, 7, 4, 1, 0, 0, 2, 1, 6, 2, 4, 0, 7, 2, 6, 5, 0, 0, 2,
        3, 4, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 6, 4, 

In [235]:
best_eps = [results[i]['best_epsilon'] for i in range(8)]
best_eps

[0.01,
 0.17244897959183675,
 0.01,
 0.01,
 0.05061224489795919,
 0.01,
 0.01,
 0.05061224489795919]

In [236]:
best_min_samples = [results[i]['best_min_samples'] for i in range(8)]
best_min_samples

[4, 2, 7, 2, 19, 2, 5, 2]

In [237]:
best_sil_score = [results[i]['best_score'] for i in range(7)]
best_sil_score

[0.49996837934115174,
 0.8686840916153955,
 0.4522821903121709,
 0.6041143534468923,
 0.6372616525530944,
 0.601677070257817,
 0.45585522605354506]

# Test
## Data Load -> Scailing -> SVD -> DBSCAN(with Best EPS, Min_samples) -> Get Label

In [238]:
test_df = pd.read_csv("./dataset/test_data.csv")
X_test = test_df[features]
y_test = test_df[['type']]

In [239]:
sc = MinMaxScaler()
X_test[features]= sc.fit_transform(X_test)
X_test.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe
0,0.367797,0.633592,1.0,0.395891,0.618443,0.599312,0.351981
1,0.393220,0.692170,1.0,0.423721,0.684856,0.653738,0.372960


In [240]:
test_df = X_test
test_df['type'] = y_test
test_df.head(2)

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,0.367797,0.633592,1.0,0.395891,0.618443,0.599312,0.351981,0
1,0.393220,0.692170,1.0,0.423721,0.684856,0.653738,0.372960,0


In [241]:
g1_test = test_df[g1]
g2_test = test_df[g2]
g3_test = test_df[g3]
y_test = test_df[['type']]
combined_test = [g1_test, g2_test, g3_test]

In [242]:
svd = NMF(n_components=1, init='random', random_state=0)
svd_list = []
for i in combined_test:
    result = svd.fit_transform(i)
    svd_list.append(result)
#     total_var =svd.explained_variance_ratio_.sum() * 100
#     print(f"{i}total_var : {total_var}")
    
flat_svd_list = []
for i in svd_list:
    result = [j for sub in i for j in sub]
    flat_svd_list.append(result)
svd_test_df = pd.DataFrame({"PC1": flat_svd_list[0], "PC2": flat_svd_list[1], "PC3": flat_svd_list[2]})
svd_test_df = pd.concat([svd_test_df, y_test], axis=1)
svd_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7389 entries, 0 to 7388
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PC1     7389 non-null   float64
 1   PC2     7389 non-null   float64
 2   PC3     7389 non-null   float64
 3   type    7389 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 231.0 KB


In [243]:
tdf0_test = svd_test_df[svd_test_df.type == 0]
tdf1_test = svd_test_df[svd_test_df.type == 1]
tdf2_test = svd_test_df[svd_test_df.type == 2]
tdf3_test = svd_test_df[svd_test_df.type == 3]
tdf4_test = svd_test_df[svd_test_df.type == 4]
tdf5_test = svd_test_df[svd_test_df.type == 5]
tdf6_test = svd_test_df[svd_test_df.type == 6]
tdf7_test = svd_test_df[svd_test_df.type == 7]
tdf_test = [tdf0_test, tdf1_test, tdf2_test, tdf3_test, tdf4_test, tdf5_test, tdf6_test, tdf7_test]

In [244]:
clusters = []
sil_scores = []

for i, tdf in enumerate(tdf_test):
    X_test = tdf.iloc[:,:-1]
    y_test = tdf[['type']]
    
    dbscan = DBSCAN(eps=best_eps[i], min_samples=best_min_samples[i]).fit(X_test)

    try: 
        sil_score = ss(X_test, dbscan.labels_)
        sil_scores.append(sil_score) 
    except:
        sil_scores.append("값없음") 
        
    clusters.extend(dbscan.labels_)

In [245]:
sil_scores

[-0.7844938344649631,
 '값없음',
 -0.1348777911907236,
 0.3631449657717098,
 0.16694815333405058,
 -0.6260165204173695,
 -0.21963596512583475,
 '값없음']

In [246]:
svd_test_df['label'] = clusters
svd_test_df['label'].value_counts()

 0     5472
 1      797
-1      196
 2      162
 6      124
 4      112
 5      102
 8       97
 3       80
 10      61
 11      49
 9       42
 7       26
 12      17
 13      17
 15       8
 14       6
 17       5
 16       4
 19       4
 18       4
 20       4
Name: label, dtype: int64

In [247]:
# ddf = svd_test_df['label'].value_counts().to_frame()
# ddf

In [248]:
# svd_test_df.info()

In [249]:
# def 결과정리(label):
#     if label in [-1, 18, 13, 12, 10, 17, 16, 11, 15, 14, 8, 9, 6, 2, 1, 5, 4, 7, 3]: 
#         return 1
#     else:
#         return 0

In [250]:
def 결과정리(label):
    if label in [0,1,2, 6, 4, 5, 8,3, 10, 11, 9]: 
        return 0
    else:
        return 1

In [251]:
svd_test_df['label'] = svd_test_df['label'].apply(결과정리)
svd_test_df['label'].value_counts()

0    7098
1     291
Name: label, dtype: int64

In [252]:
overall_sil_score = ss(svd_test_df.iloc[:,:-2], svd_test_df['label'])
overall_sil_score

-0.0049622675588992665

In [253]:
fig = px.scatter_3d(
    svd_test_df, x='PC1', y='PC2', z='PC3', color='label',symbol='label', opacity=0.5, size_max=10,height=800,
)
fig.show()

In [254]:
submit_df = svd_test_df[['type', 'label']]
submit_df['label'].value_counts()

0    7098
1     291
Name: label, dtype: int64

In [255]:
submit_df.to_csv('./submits/submit_20230411_02.csv', index=False)